In [2]:
import os
import time
import datetime

import pandas as pd
import polars as pl
import numpy as np

# Setup

In [ ]:
DATA_PATH = "../data"
RAW_DATA_DIR = os.path.join(DATA_PATH, "input", "raw")

In [ ]:
df_content = pd.read_csv(os.path.join(RAW_DATA_DIR, "content.csv"))
df_content

In [ ]:
df_topics = pd.read_csv(os.path.join(RAW_DATA_DIR, "topics.csv"))
df_topics

In [ ]:
df_correlations = pd.read_csv(os.path.join(RAW_DATA_DIR, "correlations.csv"))
df_correlations

In [ ]:
df_sample_submission = pd.read_csv(os.path.join(RAW_DATA_DIR, "sample_submission.csv"))
df_sample_submission

# Fold0

In [3]:
df_valid_topic = pd.read_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_topics_flod0_no_source.pqt")
df_valid_topic

,id,title,description,channel,category,level,language,parent,has_content
0,t_0008768bdee6,100 સુધીનો સરવાળો,37 અને 49 જેવી બે-અંકની સંખ્યાઓ ઉમેરતા શીખો.,5223e0,supplemental,4,gu,t_0da7a331d666,True
1,t_0015856edda8,Javascript,None,fc9105,supplemental,5,es,t_121a47996712,False
2,t_00165488ad43,Significance tests (hypothesis testing),Significance tests give us a formal process fo...,2ee29d,aligned,3,en,t_a7a47223471d,False
3,t_001bcbb22694,Lección 1,None,6e90a7,aligned,6,es,t_5d10d6819e04,True
4,t_00250c0c6e1a,Introduction to 3D figures,None,274269,supplemental,4,en,t_f9607325600a,False
...,...,...,...,...,...,...,...,...,...
6709,t_ffdf7dfd64fc,3 Addition,None,8bbad6,supplemental,3,en,t_a34923b85518,False
6710,t_fff05585df72,11: Systems of Equations and Inequalities,None,1fb613,supplemental,4,en,t_5ab3d2eac617,True
6711,t_fff51448598c,Comparative advantage and the gains from trade,Comparative advantage is the ability to produc...,2ee29d,aligned,4,en,t_9d906e167aea,True
6712,t_fff80f4eee89,Polynomial division,"After we have added, subtracted, and multiplie...",6b09a4,supplemental,5,en,t_e93f4fad3893,False


In [4]:
df_valid_label = pd.read_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_correlations_flod0.pqt")
df_valid_label

,topic_id,content_ids
0,t_00068291e9a4,c_639ea2ef9c95
1,t_00068291e9a4,c_89ce9367be10
2,t_00068291e9a4,c_ac1672cdcd2c
3,t_00068291e9a4,c_ebb7fdf10a7e
4,t_0008768bdee6,c_34e1424229b4
...,...,...
55073,t_fff51448598c,c_ab59efb09f73
55074,t_fff51448598c,c_cef3d40736d3
55075,t_fff51448598c,c_d706cb35bd02
55076,t_fff51448598c,c_e346107f23d9


In [5]:
df_valid_label = df_valid_label[df_valid_label["topic_id"].isin(df_valid_topic["id"].unique())]
df_valid_label

,topic_id,content_ids
4,t_0008768bdee6,c_34e1424229b4
5,t_0008768bdee6,c_7d1a964d66d5
6,t_0008768bdee6,c_aab93ee667f4
12,t_001bcbb22694,c_1d9dfc709413
22,t_002dfcaaf1d7,c_7c35d77064e5
...,...,...
55073,t_fff51448598c,c_ab59efb09f73
55074,t_fff51448598c,c_cef3d40736d3
55075,t_fff51448598c,c_d706cb35bd02
55076,t_fff51448598c,c_e346107f23d9


In [6]:
df_valid_label.reset_index(drop=True).to_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_correlations_flod0_no_source.pqt")